## NYT COVID-19 Analysis
This notebook processes and performs quick analysis from the [New York Times COVID-19 dataset](https://github.com/nytimes/covid-19-data).  The data is updated in the `/databricks-datasets/COVID/covid-19-data/` location regularly so you can access the data directly.

In [2]:
# Standard Libraries
import io

# External Libraries
import requests
import numpy as np
import pandas as pd
import altair as alt
from vega_datasets import data

# topographical
topo_usa = 'https://vega.github.io/vega-datasets/data/us-10m.json'
topo_wa = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/WA-53-washington-counties.json'
topo_king = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/WA/King.geo.json'

### Download Mapping County FIPS to lat, long_

In [4]:
%sh mkdir -p /dbfs/tmp/dennylee/COVID/map_fips/ && wget -O /dbfs/tmp/dennylee/COVID/map_fips/countyfips_lat_long.csv https://raw.githubusercontent.com/dennyglee/tech-talks/master/datasets/countyfips_lat_long.csv && ls -al /dbfs/tmp/dennylee/COVID/map_fips/

--2020-04-13 23:01:06-- https://raw.githubusercontent.com/dennyglee/tech-talks/master/datasets/countyfips_lat_long.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129178 (126K) [text/plain]
Saving to: ‘/dbfs/tmp/dennylee/COVID/map_fips/countyfips_lat_long.csv’

 0K .......... .......... .......... .......... .......... 39% 2.22M 0s
 50K .......... .......... .......... .......... .......... 79% 5.31M 0s
 100K .......... .......... ...... 100% 11.8M=0.03s

2020-04-13 23:01:06 (3.69 MB/s) - ‘/dbfs/tmp/dennylee/COVID/map_fips/countyfips_lat_long.csv’ saved [129178/129178]

total 135
drwxrwxrwx 2 root root 4096 Apr 13 2020 .
drwxrwxrwx 2 root root 4096 Apr 13 21:13 ..
-rwxrwxrwx 1 root root 129178 Apr 13 2020 countyfips_lat_long.csv

In [5]:
# Create mapping of county FIPS to centroid long_ and lat
map_fips = spark.read.option("header", True).option("inferSchema", True).csv("/tmp/dennylee/COVID/map_fips/countyfips_lat_long.csv")
map_fips = (map_fips
              .withColumnRenamed("STATE", "state")
              .withColumnRenamed("COUNTYNAME", "county")
              .withColumnRenamed("LAT", "lat")
              .withColumnRenamed("LON", "long_"))
map_fips.createOrReplaceTempView("map_fips")

In [6]:
map_fips_dedup = spark.sql("""select fips, min(state) as state, min(county) as county, min(long_) as long_, min(lat) as lat from map_fips group by fips""")
map_fips_dedup.createOrReplaceTempView("map_fips_dedup")

### Download 2014 Population Estimates

In [8]:
%sh mkdir -p /dbfs/tmp/dennylee/COVID/population_estimates_by_county/ && wget -O /dbfs/tmp/dennylee/COVID/population_estimates_by_county/CO-EST2014-alldata.csv https://raw.githubusercontent.com/dennyglee/tech-talks/master/datasets/county-estimates.csv && ls -al /dbfs/tmp/dennylee/COVID/population_estimates_by_county/

--2020-04-14 01:21:55-- https://raw.githubusercontent.com/dennyglee/tech-talks/master/datasets/county-estimates.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1781101 (1.7M) [text/plain]
Saving to: ‘/dbfs/tmp/dennylee/COVID/population_estimates_by_county/CO-EST2014-alldata.csv’

 0K .......... .......... .......... .......... .......... 2% 3.52M 0s
 50K .......... .......... .......... .......... .......... 5% 8.06M 0s
 100K .......... .......... .......... .......... .......... 8% 25.9M 0s
 150K .......... .......... .......... .......... .......... 11% 11.8M 0s
 200K .......... .......... .......... .......... .......... 14% 29.7M 0s
 250K .......... .......... .......... .......... .......... 17% 29.4M 0s
 300K .......... .......... .......... .......... .......... 20% 31.0M 0s
 350K .......... .......... .......... .......... .......... 22% 18.3M 0s
 400K .......... .......... .......... .......... .......... 25% 26.5M 0s
 450K .......... .......... .......... .......... .......... 28% 29.6M 0s
 500K .......... .......... .......... .......... .......... 31% 31.2M 0s
 550K .......... .......... .......... .......... .......... 34% 25.4M 0s
 600K .......... .......... .......... .......... .......... 37% 28.0M 0s
 650K .......... .......... .......... .......... .......... 40% 30.6M 0s
 700K .......... .......... .......... .......... .......... 43% 30.5M 0s
 750K .......... .......... .......... .......... .......... 45% 21.4M 0s
 800K .......... .......... .......... .......... .......... 48% 29.5M 0s
 850K .......... .......... .......... .......... .......... 51% 30.7M 0s
 900K .......... .......... .......... .......... .......... 54% 26.9M 0s
 950K .......... .......... .......... .......... .......... 57% 25.9M 0s
 1000K .......... .......... .......... .......... .......... 60% 28.0M 0s
 1050K .......... .......... .......... .......... .......... 63% 30.7M 0s
 1100K .......... .......... .......... .......... .......... 66% 31.6M 0s
 1150K .......... .......... .......... .......... .......... 68% 24.8M 0s
 1200K .......... .......... .......... .......... .......... 71% 30.1M 0s
 1250K .......... .......... .......... .......... .......... 74% 29.9M 0s
 1300K .......... .......... .......... .......... .......... 77% 31.0M 0s
 1350K .......... .......... .......... .......... .......... 80% 24.7M 0s
 1400K .......... .......... .......... .......... .......... 83% 29.3M 0s
 1450K .......... .......... .......... .......... .......... 86% 28.6M 0s
 1500K .......... .......... .......... .......... .......... 89% 30.8M 0s
 1550K .......... .......... .......... .......... .......... 91% 26.3M 0s
 1600K .......... .......... .......... .......... .......... 94% 32.1M 0s
 1650K .......... .......... .......... .......... .......... 97% 31.8M 0s
 1700K .......... .......... .......... ......... 100% 30.5M=0.08s

2020-04-14 01:21:55 (21.4 MB/s) - ‘/dbfs/tmp/dennylee/COVID/population_estimates_by_county/CO-EST2014-alldata.csv’ saved [1781101/1781101]

total 1748
drwxrwxrwx 2 root root 4096 Apr 14 2020 .
drwxrwxrwx 2 root root 4096 Apr 14 01:17 ..
-rwxrwxrwx 1 root root 1781101 Apr 14 2020 CO-EST2014-alldata.csv

In [9]:
map_popest_county = spark.read.option("header", True).option("inferSchema", True).csv("/tmp/dennylee/COVID/population_estimates_by_county/CO-EST2014-alldata.csv")
map_popest_county.createOrReplaceTempView("map_popest_county")
fips_popest_county = spark.sql("select State * 1000 + substring(cast(1000 + County as string), 2, 3) as fips, STNAME, CTYNAME, census2010pop, POPESTIMATE2014 from map_popest_county")
fips_popest_county.createOrReplaceTempView("fips_popest_county")

## Specify `nyt_daily` table
* Source: `/databricks-datasets/COVID/covid-19-data/`
* Contains the COVID-19 daily reports

In [11]:
nyt_daily = spark.read.option("inferSchema", True).option("header", True).csv("/databricks-datasets/COVID/covid-19-data/us-counties.csv")
nyt_daily.createOrReplaceTempView("nyt_daily")
display(nyt_daily)

date,county,state,fips,cases,deaths
2020-01-21T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-22T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-23T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-24T00:00:00.000+0000,Cook,Illinois,17031,1,0
2020-01-24T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-25T00:00:00.000+0000,Orange,California,6059,1,0
2020-01-25T00:00:00.000+0000,Cook,Illinois,17031,1,0
2020-01-25T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-26T00:00:00.000+0000,Maricopa,Arizona,4013,1,0
2020-01-26T00:00:00.000+0000,Los Angeles,California,6037,1,0


# COVID-19 Cases and Deaths for Specific Counties
Focusing on two-week window around when educational facilites were closed
* Top 10 Washington State counties (3/13/2020)
* Top 10 NY State counties (3/18/2020)

In [13]:
# WA State 2 week window
wa_state_window = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-06T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  from (
SELECT CAST(f.date AS date) AS date, f.county, f.fips, SUM(f.cases) AS cases, SUM(f.deaths) AS deaths, MAX(p.POPESTIMATE2014) AS population_estimate 
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = f.fips
 WHERE f.state = 'Washington' 
   AND date BETWEEN '2020-03-06T00:00:00.000+0000' AND '2020-03-20T00:00:00.000+0000'
 GROUP BY f.date, f.county, f.fips
) a""")
wa_state_window.createOrReplaceTempView("wa_state_window")

# NY State 2 week window
ny_state_window = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-11T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  FROM (
SELECT CAST(f.date AS date) AS date, f.county, p.fips, SUM(f.cases) as cases, SUM(f.deaths) as deaths, MAX(p.POPESTIMATE2014) AS population_estimate  
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = coalesce(f.fips, 36061)
 WHERE f.state = 'New York' 
   AND date BETWEEN '2020-03-11T00:00:00.000+0000' AND '2020-03-25T00:00:00.000+0000'
 GROUP BY f.date, f.county, p.fips
) a""")
ny_state_window.createOrReplaceTempView("ny_state_window")

# NY State 2 week window (-1 week)
ny_state_window_m1 = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-06T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  FROM (
SELECT CAST(f.date AS date) AS date, f.county, p.fips, SUM(f.cases) as cases, SUM(f.deaths) as deaths, MAX(p.POPESTIMATE2014) AS population_estimate  
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = coalesce(f.fips, 36061)
 WHERE f.state = 'New York' 
   AND date BETWEEN '2020-03-06T00:00:00.000+0000' AND '2020-03-20T00:00:00.000+0000'
 GROUP BY f.date, f.county, p.fips
) a""")
ny_state_window_m1.createOrReplaceTempView("ny_state_window_m1")

## COVID-19 Cases for WA and NY Counties

In [15]:
%sql
SELECT f.date, f.county, f.cases 
  FROM wa_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM wa_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases
2020-03-11,Island,1
2020-03-11,King,235
2020-03-12,Pierce,17
2020-03-13,Snohomish,132
2020-03-20,Island,19
2020-03-07,Snohomish,26
2020-03-15,Kitsap,5
2020-03-12,Spokane,4
2020-03-12,Snohomish,107
2020-03-15,Whatcom,3


In [16]:
%sql
SELECT f.date, f.county, f.cases 
  FROM ny_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases
2020-03-17,Suffolk,84
2020-03-20,Erie,31
2020-03-17,Westchester,379
2020-03-24,Suffolk,1880
2020-03-13,Monroe,1
2020-03-14,Westchester,178
2020-03-14,Albany,5
2020-03-19,New York City,2468
2020-03-16,Orange,11
2020-03-12,Monroe,1


In [17]:
%sql
SELECT f.date, f.county, f.cases 
  FROM ny_state_window_m1 f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window_m1 GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases
2020-03-17,Suffolk,84
2020-03-11,Nassau,28
2020-03-17,Westchester,379
2020-03-08,Westchester,81
2020-03-08,Suffolk,1
2020-03-13,Monroe,1
2020-03-14,Westchester,178
2020-03-08,Nassau,5
2020-03-14,Albany,5
2020-03-19,New York City,2468


## COVID-19 Cases per 100K people for WA and NY Counties
Let's look at these values by a percentage of the population; the numbers used are the 2014 US Census estimates of county populations.

*Note, reviewing the top 10 counties by case (vs. % of cases)*

In [19]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM wa_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM wa_state_window GROUP BY county ORDER BY cases DESC LIMIT 10  
    ) x ON x.county = f.county

date,county,cases_per_100Kpop
2020-03-11,Island,1.26143172501
2020-03-11,King,11.29825617426
2020-03-12,Pierce,2.04344606745
2020-03-13,Snohomish,17.37795606273
2020-03-20,Island,23.96720277515
2020-03-07,Snohomish,3.42293073963
2020-03-15,Kitsap,1.96708670525
2020-03-12,Spokane,0.82590364182
2020-03-12,Snohomish,14.0866765054
2020-03-15,Whatcom,1.43987789835


In [20]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM ny_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases_per_100Kpop
2020-03-17,Suffolk,5.58894134805
2020-03-20,Erie,3.35921372726
2020-03-17,Westchester,38.96635322228
2020-03-24,Suffolk,125.0858301707
2020-03-13,Monroe,0.1333587604
2020-03-14,Westchester,18.30082024688
2020-03-14,Albany,1.62247583322
2020-03-19,New York City,150.83103745841
2020-03-16,Orange,2.92476183133
2020-03-12,Monroe,0.1333587604


In [21]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM ny_state_window_m1 f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window_m1 GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases_per_100Kpop
2020-03-17,Suffolk,5.58894134805
2020-03-11,Nassau,2.06090413336
2020-03-17,Westchester,38.96635322228
2020-03-08,Westchester,8.3279013483
2020-03-08,Suffolk,0.06653501605
2020-03-13,Monroe,0.1333587604
2020-03-14,Westchester,18.30082024688
2020-03-08,Nassau,0.36801859524
2020-03-14,Albany,1.62247583322
2020-03-19,New York City,150.83103745841


## Visualize Cases by State Choropleth Maps
* Join the data with `map_fips_dedup` to obtain the county centroid lat, long_

In [23]:
%sql
select * from wa_state_window limit 10

date,day_num,county,fips,cases,deaths,cases_per_100Kpop,deaths_per_100Kpop
2020-03-11,5,Island,53029,1,0,1.26143172501,0.0
2020-03-11,5,King,53033,235,27,11.29825617426,1.29809751789
2020-03-12,6,Pierce,53053,17,0,2.04344606745,0.0
2020-03-13,7,Snohomish,53061,132,3,17.37795606273,0.39495354688
2020-03-15,9,Columbia,53013,1,0,25.09410288582,0.0
2020-03-13,7,Jefferson,53031,1,0,3.30819108112,0.0
2020-03-14,8,Grant,53025,1,0,1.07357188101,0.0
2020-03-20,14,Grays Harbor,53027,1,0,1.41207037759,0.0
2020-03-20,14,Kittitas,53037,4,0,9.40689525422,0.0
2020-03-20,14,Stevens,53065,1,0,2.29095074456,0.0


In [24]:
# Extract Day Number and county centroid lat, long_
wa_daynum = spark.sql("""select f.fips, f.county, f.day_num, cast(f.cases_per_100Kpop as int) as confirmed, cast(f.deaths_per_100Kpop as int) as deaths, m.lat, m.long_ from wa_state_window f join map_fips_dedup m on m.fips = f.fips""")
wa_daynum.createOrReplaceTempView("wa_daynum")
ny_daynum = spark.sql("""select cast(f.fips as int) as fips, f.county, f.day_num, cast(f.cases_per_100Kpop as int) as confirmed, cast(f.deaths_per_100Kpop as int) as deaths, m.lat, m.long_ from ny_state_window f join map_fips_dedup m on m.fips = f.fips""")
ny_daynum.createOrReplaceTempView("ny_daynum")
ny_daynum_m1 = spark.sql("""select cast(f.fips as int) as fips, f.county, f.day_num, cast(f.cases_per_100Kpop as int) as confirmed, cast(f.deaths_per_100Kpop as int) as deaths, m.lat, m.long_ from ny_state_window_m1 f join map_fips_dedup m on m.fips = f.fips""")
ny_daynum_m1.createOrReplaceTempView("ny_daynum_m1")

In [25]:
# Obtain Topography
topo_usa = 'https://vega.github.io/vega-datasets/data/us-10m.json'
topo_wa = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/WA-53-washington-counties.json'
topo_ny = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/NY-36-new-york-counties.json'
us_counties = alt.topo_feature(topo_usa, 'counties')
wa_counties = alt.topo_feature(topo_wa, 'cb_2015_washington_county_20m')
ny_counties = alt.topo_feature(topo_ny, 'cb_2015_new_york_county_20m')

In [26]:
# Review WA
confirmed_wa = wa_daynum.select("fips", "day_num", "confirmed", "county").where("confirmed > 0").toPandas()
deaths_wa = wa_daynum.select("lat", "long_", "day_num", "deaths", "county").where("deaths > 0").toPandas()

# Review NY
confirmed_ny = ny_daynum.select("fips", "day_num", "confirmed", "county").where("confirmed > 0").toPandas()
deaths_ny = ny_daynum.select("lat", "long_", "day_num", "deaths", "county").where("deaths > 0").toPandas()

In [27]:
# State Choropleth Map Visualization Function
def map_state(curr_day_num, state_txt, state_counties, confirmed, confirmed_min, confirmed_max, deaths, deaths_min, deaths_max):
  # Washington State
  base_state = alt.Chart(state_counties).mark_geoshape(
      fill='white',
      stroke='lightgray',
  ).properties(
      width=1000,
      height=800,
  ).project(
      type='mercator'
  )

  # counties
  base_state_counties = alt.Chart(us_counties).mark_geoshape(
  ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(confirmed[(confirmed['confirmed'] > 0) & (confirmed['day_num'] == curr_day_num)], 'fips', ['confirmed', 'county'])  
  ).encode(
     color=alt.Color('confirmed:Q', scale=alt.Scale(type='log', domain=[confirmed_min, confirmed_max]), title='Confirmed'),
    tooltip=[
      alt.Tooltip('confirmed:Q'),
      alt.Tooltip('county:O'),
    ],
  )

  # deaths by long, latitude
  points = alt.Chart(deaths[(deaths['deaths'] > 0) & (deaths['day_num'] == curr_day_num)]).mark_point(opacity=0.75, filled=True).encode(
    longitude='long_:Q',
    latitude='lat:Q',
    size=alt.Size('sum(deaths):Q', scale=alt.Scale(type='symlog', domain=[deaths_min, deaths_max]), title='deaths'),
    color=alt.value('#BD595D'),
    stroke=alt.value('brown'),
    tooltip=[
      alt.Tooltip('lat'),
      alt.Tooltip('long_'),
      alt.Tooltip('deaths'),
      alt.Tooltip('county:O'),      
    ],
  ).properties(
    # update figure title
    title=f'COVID-19 {state_txt} Confirmed Cases and Deaths per 100K by County [{curr_day_num}]'
  )

  return (base_state + base_state_counties + points)


| Factors | WA | NY | 
| ------- | -- | -- | 
| Educational Facilities Closed | 3/13/2020 | 3/18/2020 |
| Day 00 | 3/6/2020 | 3/11/2020 |
| Day 14 | 3/20/2020 | 3/25/2020 | 
| Max Cases | 50.55 | 1222.97 | 
| Max Deaths | 3.27 | 17.11 |

### WA State

In [30]:
map_state(101, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

In [31]:
map_state(107, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

In [32]:
map_state(114, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

### NY State

In [34]:
map_state(101, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

In [35]:
map_state(107, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

In [36]:
map_state(114, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

## COVID-19 Confirmed Cases and Deaths by WA and NY County Slider

In [38]:
# pivot confirmed cases
confirmed_pv_wa = confirmed_wa[['fips', 'day_num', 'confirmed']]
confirmed_pv_wa['fips'] = confirmed_pv_wa['fips'].astype(str)
confirmed_pv_wa['day_num'] = confirmed_pv_wa['day_num'].astype(str)
confirmed_pv_wa['confirmed'] = confirmed_pv_wa['confirmed'].astype('int64')
confirmed_pv_wa = confirmed_pv_wa.pivot_table(index='fips', columns='day_num', values='confirmed', fill_value=0).reset_index()

# pivot deaths
deaths_pv_wa = deaths_wa[['lat', 'long_', 'day_num', 'deaths']]
deaths_pv_wa['day_num'] = deaths_pv_wa['day_num'].astype(str)
deaths_pv_wa['deaths'] = deaths_pv_wa['deaths'].astype('int64')
deaths_pv_wa = deaths_pv_wa.pivot_table(index=['lat', 'long_'], columns='day_num', values='deaths', fill_value=0).reset_index()

# Extract column names for slider
column_names = confirmed_pv_wa.columns.tolist()

# Remove first element (`fips`)
column_names.pop(0)

# Convert to int
column_values = [None] * len(column_names)
for i in range(0, len(column_names)): column_values[i] = int(column_names[i]) 

/local_disk0/tmp/1586814074438-0/PythonShell.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 
/local_disk0/tmp/1586814074438-0/PythonShell.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import errno
/local_disk0/tmp/1586814074438-0/PythonShell.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import linecache
/local_disk0/tmp/1586814074438-0/PythonShell.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import six
/local_disk0/tmp/1586814074438-0/PythonShell.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import six.moves.queue

In [39]:
# Disable max_rows to see more data
alt.data_transformers.disable_max_rows()

# Topographic information
us_states = alt.topo_feature(topo_usa, 'states')
us_counties = alt.topo_feature(topo_usa, 'counties')

# state county boundaries
base_state = alt.Chart(wa_counties).mark_geoshape(
    fill='white',
    stroke='lightgray',
).properties(
    width=1000,
    height=800,
).project(
    type='mercator'
)

# Slider choices
min_day_num = column_values[0]
max_day_num = column_values[len(column_values)-1]
slider = alt.binding_range(min=min_day_num, max=max_day_num, step=1)
slider_selection = alt.selection_single(fields=['day_num'], bind=slider, name="day_num", init={'day_num':min_day_num})


# Confirmed cases by county
base_state_counties = alt.Chart(us_counties).mark_geoshape(
    stroke='black',
    strokeWidth=0.05
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(confirmed_pv_wa, 'fips', column_names)  
).transform_fold(
    column_names, as_=['day_num', 'confirmed']
).transform_calculate(
    state_id = "(datum.id / 1000)|0",
    day_num = 'parseInt(datum.day_num)',
    confirmed = 'isValid(datum.confirmed) ? datum.confirmed : -1'
).encode(
    color = alt.condition(
        'datum.confirmed > 0',      
        alt.Color('confirmed:Q', scale=alt.Scale(domain=(1, 60), type='symlog')),
        alt.value('white')
      )  
).properties(
  # update figure title
  title=f'COVID-19 WA State Confirmed Cases per 100K by County'
).transform_filter(
    (alt.datum.state_id)==53
# ).add_selection(
#     slider_selection
).transform_filter(
    slider_selection
)


# deaths by long, latitude
points = alt.Chart(
  deaths_pv_wa
).mark_point(
  opacity=0.75, filled=True
).transform_fold(
  column_names, as_=['day_num', 'deaths']
).transform_calculate(
    day_num = 'parseInt(datum.day_num)',
    deaths = 'isValid(datum.deaths) ? datum.deaths : -1'  
).encode(
  longitude='long_:Q',
  latitude='lat:Q',
  size=alt.Size('deaths:Q', scale=alt.Scale(domain=(1, 5), type='symlog'), title='deaths'),
  color=alt.value('#BD595D'),
  stroke=alt.value('brown'),
).add_selection(
    slider_selection
).transform_filter(
    slider_selection
)

# confirmed cases (base_counties) and deaths (points)
(base_state + base_state_counties + points) 